In [117]:
from codebase.classes_ibis_lvm import ParticlesLVM
from codebase.classes_data import Data
from codebase.ibis import model_phonebook, essl
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis import run_ibis

from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
from codebase.plot import plot_density, get_post_df, plot_correlations
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## 2 factor Sim EZ

In [206]:
log_dir = './log/20210101_205234_test/'

## Create Data

In [207]:
data = load_obj('data', log_dir)

In [214]:
data.raw_data

{'random_seed': 0,
 'N': 30,
 'J': 6,
 'K': 1,
 'alpha': array([-0.53,  0.35, -1.4 , -1.4 , -0.96, -2.33]),
 'beta': array([1. , 0.7, 0.8, 0.5, 0.9, 0.6]),
 'z': array([ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
        -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ,
         0.14404357,  1.45427351,  0.76103773,  0.12167502,  0.44386323,
         0.33367433,  1.49407907, -0.20515826,  0.3130677 , -0.85409574,
        -2.55298982,  0.6536186 ,  0.8644362 , -0.74216502,  2.26975462,
        -1.45436567,  0.04575852, -0.18718385,  1.53277921,  1.46935877]),
 'y': array([[ 1.23405235,  1.58483664,  0.01124188, -0.51797383,  0.62764711,
         -1.27156859],
        [-0.12984279,  0.63011005, -1.07987423, -1.1999214 , -0.59985851,
         -2.08990567],
        [ 0.44873798,  1.03511659, -0.61700961, -0.91063101, -0.07913581,
         -1.74275721],
        [ 1.7108932 ,  1.91862524,  0.39271456, -0.2795534 ,  1.05680388,
         -0.98546408],
      

## Run HMC

In [151]:
param_names = ['beta', 'alpha', 'betabeta']
ps= load_obj( 'ps_hmc', log_dir)

In [152]:
np.round(np.mean(ps['beta'], 0),2)

array([[0.59, 1.63, 0.72, 0.82, 0.87, 0.97]])

In [153]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]).reshape(1000,6,1)) 
df['source'] = 'hmc'
plot_density(df) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.Chart(...)

## Run IBIS

## Plot Both results

In [209]:
particles = load_obj('particles', log_dir)
print(load_obj('t', log_dir))

29


In [200]:
# have to resample particles to get rid of weights
particles.resample_particles()

In [211]:
np.mean(np.squeeze(particles.particles['beta']),0)

array([ 0.0450356 , -0.13135953,  0.17357059, -0.27895913,  0.07594751,
        0.08492647])

In [212]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]).reshape(1000,6,1)) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.Chart(...)

In [189]:
param = 'alpha'
df = get_post_df(np.squeeze(ps[param])) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'
plot_density(pd.concat([df,df2])) 
# &\
# plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.Chart(...)